In [15]:
import os

In [16]:
VERBOSE = str(os.getenv("YOLO_VERBOSE", True)).lower() == "False"  # global verbose mode

In [17]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [18]:
import ultralytics
ultralytics.__version__

'8.3.9'

In [19]:
# Not training the model, just using pretrained model trained on COCO dataset. 
# Aquest dataset té som vehicles a part d'altres objectes.
#Fer servir aquest model per la detecció i després track i count
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import *
import os


os.environ["YOLO_VERBOSE"] = "false"


model=YOLO('yolov5n.pt')

PRO TIP  Replace 'model=yolov5n.pt' with new 'model=yolov5nu.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.



In [20]:
#list of COCO dataset classes
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [21]:
#creant una instancia de tracker que ve de from tracker import*
tracker=Tracker()
count=0

In [22]:
#llegir el video
cap=cv2.VideoCapture('output2.mp4')

In [25]:
#VARIABLES PER GUARDAR COTXES UP I DOWN
down={}
up={}

counter_down=[]
counter_up=[]
downwards = 0
upwards = 0

In [26]:
import time
import numpy as np

# Initialize the FPS counter
fps = 0
start_time = time.time()
pts_ROI = np.array([[359, 480],[186, 480] ,[8, 840], [446, 841]], np.int32)
# divide all values by 2
pts_ROI = pts_ROI // 2
pts_ROI = pts_ROI.reshape((-1, 1, 2))

frame_difference = np.array([])

pts = np.array([[359, 340],[186, 371] ,[8, 953], [446, 951]], np.int32)
# divide all values by 2
pts = pts // 2
pts = pts.reshape((-1, 1, 2))

last_frame = None

while True:
    skip = False
    ret, frame = cap.read()
    if not ret:
        break
    count += 1
    # RESOLUCIO IMATGE ORIGINAL: 960X540
    # BAIXAR RESOLUCIO
    frame = cv2.resize(frame, (270, 480))
    original_frame = frame.copy()

    mask = np.zeros_like(frame, dtype=np.uint8)
    cv2.fillPoly(mask, [pts], (255, 255, 255))
    roi_frame = cv2.bitwise_and(original_frame, mask)
    
    if last_frame is not None:
        frame_difference = cv2.absdiff(last_frame, frame)*2


        frame_difference = cv2.bitwise_and(frame_difference, mask)

        print(frame_difference.mean())

        cv2.imshow("frame_difference", frame_difference)

    # Saltar 1 de cada 2 frames i quan la mascara no hi ha yolo
    if count % 2 != 0 or frame_difference.mean() < 1:
        fps += 1
        end_time = time.time()
        elapsed_time = end_time - start_time
        if elapsed_time > 0:
            current_fps = fps / elapsed_time
            cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.line(frame, (0, 400), (800, 400), (0, 0, 255), 3)  # red line
            cv2.line(frame, (0, 450), (850, 450), (255, 0, 0), 3)  # blue line
            cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            

    # Convertir el frame a grisos per anar més ràpid: 
    # No es pot utilitzar perquè YOLO treballa en RGB
    # frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)


    else:
        # YOLO Prediction
        results = model.predict(roi_frame)

        # aconseguint les dades de la bounding box i confiança
        a = results[0].boxes.data
        a = a.detach().cpu().numpy()
        px = pd.DataFrame(a).astype("float")

        # only work with car class. only count cars. Quedarnos bounding box dels cars
        car_bboxes = []
        for index, row in px.iterrows():
            x1 = int(row[0])
            y1 = int(row[1])
            x2 = int(row[2])
            y2 = int(row[3])
            d = int(row[5])
            c = class_list[d]
            if 'car' in c:
                car_bboxes.append([x1, y1, x2, y2])

        # provide les coordinades al tracker
        bbox_id = tracker.update(car_bboxes)

        # draw bbox
        for bbox in bbox_id:
            x3, y3, x4, y4, id = bbox
            cx = int((x3 + x4) // 2)
            cy = int((y3 + y4) // 2)

            # Draw bounding box and ID
            # cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
            # cv2.putText(frame, f'ID: {id}', (x3, y3 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Update down/up counters
            red_line_y = 400
            blue_line_y = 450
            offset = 7

            # DOWNWARDS condition: red line first, then blue line
            if red_line_y < (cy + offset) and red_line_y > (cy - offset):
                down[id] = cy
            if id in down:
                if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # red dot
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_down.append(id)  # list

            # UPWARDS condition: blue line first, then red line
            if blue_line_y < (cy + offset) and blue_line_y > (cy - offset):
                up[id] = cy
            if id in up:
                if red_line_y < (cy + offset) and red_line_y > (cy - offset):
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # red dot
                    cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    counter_up.append(id)  # list

        # Draw lines
        cv2.line(frame, (0, 400), (800, 400), (0, 0, 255), 3)  # red line
        cv2.line(frame, (0, 450), (850, 450), (255, 0, 0), 3)  # blue line

        # Show counters on the frame
        downwards = len(set(counter_down))
        upwards = len(set(counter_up))



        # FPS calculation
        fps += 1
        end_time = time.time()
        elapsed_time = end_time - start_time
        if elapsed_time > 0:
            current_fps = fps / elapsed_time
            cv2.putText(frame, f'FPS: {round(current_fps, 2)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"YOLO ACTIVAT", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"BAIXADA: {downwards}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
            cv2.putText(frame, f"PUJADA: {upwards}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        
    cv2.imshow("frames", frame)
    # record to video

    
    if cv2.waitKey(1) & 0xFF == 27:
        break
    
    last_frame = original_frame

cap.release()
cv2.destroyAllWindows()


C:\Users\marcp\AppData\Local\Temp\ipykernel_27784\1398700324.py:47: RuntimeWarning: Mean of empty slice.
  if count % 2 != 0 or frame_difference.mean() < 1:
c:\Users\marcp\anaconda3\Lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



0: 640x384 (no detections), 140.0ms
Speed: 10.0ms preprocess, 140.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 384)
0.5790020576131687
0.48694958847736625
0.2588940329218107
0.4084979423868313
0.28717592592592595
0.43335390946502056
0.36030864197530865
0.15430041152263374
0.1777932098765432
0.1207664609053498
0.8383641975308642
0.2539917695473251
0.1593724279835391
0.4948405349794239
0.40825102880658437
0.16627572016460906
0.08650720164609053
0.13112654320987654
0.22174382716049382
0.09095164609053498
0.32465020576131687
0.34554012345679014
0.843261316872428
0.5118364197530865
0.33420267489711936
0.3859567901234568
0.2334104938271605
0.28565329218106994
0.35061213991769546
0.1513940329218107
0.22553497942386833
0.1680812757201646
0.1395627572016461
0.1051954732510288
0.785
0.33625514403292184
0.339429012345679
0.20148148148148148
0.3132201646090535
0.2198045267489712
0.2502469135802469
0.2035082304526749
0.13006687242798354
0.2799897119341564
0.21719135802469136
0.2

In [14]:
cap.release()
cv2.destroyAllWindows()